# Point cloud visualization

## Introduction
In this tutorial, we present easy ways to visualize several different point cloud file formats that are commonly used to store point cloud-type information using two very popular python packages ([Open3D](http://www.open3d.org/) & [pptk - Point Processing Toolkit](https://github.com/heremaps/pptk)).

These are the most popular formats for point cloud data and the packages we'll use to open the files.
  - ply --> (Open3D) 
  - pcd --> (Open3D)
  - npz, npy --> (Open3D)
  - hdf5 --> (pptk)
  - binary --> (pptk)
  - las, laz --> (pptk)
  - txt --> (pptk, Open3D)

## Import the libraries

In [33]:
# Data read & write
import numpy as np
import laspy
import h5py
# Visualization
import open3d as o3d
import pptk # works with Python 3.6
import lazrs

### Format: ply | Package: Open3D

In [34]:
# Load ply file
pc_ply = o3d.io.read_point_cloud("./data/fragment.ply")
print('PLY file loaded')

PLY file loaded


In [8]:
# PLY file info
print(dir(pc_ply))

['HalfEdgeTriangleMesh', 'Image', 'LineSet', 'PointCloud', 'RGBDImage', 'TetraMesh', 'TriangleMesh', 'Type', 'Unspecified', 'VoxelGrid', '__add__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__iadd__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'cluster_dbscan', 'colors', 'compute_convex_hull', 'compute_mahalanobis_distance', 'compute_mean_and_covariance', 'compute_nearest_neighbor_distance', 'compute_point_cloud_distance', 'covariances', 'create_from_depth_image', 'create_from_rgbd_image', 'crop', 'dimension', 'estimate_covariances', 'estimate_normals', 'estimate_point_covariances', 'get_axis_aligned_bounding_box', 'get_center', 'get_geometry_type', 'get_max_bound', 'get_min_bound', 'get_oriented_bounding_box', 'get_rota

In [9]:
print('Shape of points', np.asarray(pc_ply.points).shape)
print('Shape of colors', np.asarray(pc_ply.colors).shape)

Shape of points (196133, 3)
Shape of colors (196133, 3)


In [10]:
# visualization
o3d.visualization.draw_geometries([pc_ply])
# Capture 
# o3d.visualization.Visualizer.capture_screen_image('screenshot.png')

### Format: pcd | Package: Open3D

In [11]:
# Load pcd file
pc_pcd = o3d.io.read_point_cloud("./data/pcl_CSite1_orig-utm.pcd")
# Point Cloud info
print(pc_pcd)
print('Shape of points', np.asarray(pc_pcd.points).shape)
print('Shape of colors', np.asarray(pc_pcd.colors).shape) # no color data

PointCloud with 1366408 points.
Shape of points (1366408, 3)
Shape of colors (0, 3)


In [12]:
# visualization
o3d.visualization.draw_geometries([pc_pcd])

### Format: numpy | Package: Open3D

In [13]:
pc_np = np.load('./data/shapenet_04554684-fcc0bdba1a95be2546cde67a6a1ea328.npy')
print('Shape of the Point Cloud:', pc_np.shape)
# Create PointCloud class
pc = o3d.geometry.PointCloud()
pc.points = o3d.utility.Vector3dVector(pc_np)

Shape of the Point Cloud: (8192, 3)


In [14]:
# visualization
o3d.visualization.draw_geometries([pc])

### Format: las & laz | Package: PPTK

In [12]:
point_cloud = laspy.read('./data/vegetation_1_3.las')
print(point_cloud)

<LasData(1.3, point fmt: <PointFormat(1, 0 bytes of extra dims)>, 10683 points, 0 vlrs)>


In [27]:
## lists column names in las file.
point_format = point_cloud.point_format
list(point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'synthetic',
 'key_point',
 'withheld',
 'scan_angle_rank',
 'user_data',
 'point_source_id',
 'gps_time']

In [28]:
# preparation for visualization
points = np.vstack((point_cloud.x, point_cloud.y, point_cloud.z)).transpose()
print(points.shape)

(10683, 3)


In [40]:
vis = pptk.viewer(points) # Start the interactive visualization

In [41]:
## (Optional)
vis.attributes(point_cloud['intensity'], point_cloud['classification']) # Loads point attributes (intensity) to color the points
# a few auxiliary property
vis.set(point_size=0.01)
# vis.set(bg_color=[0, 0, 0, 1])
# vis.set(show_grid=False)
# vis.set(show_axis=False)
# set color map to gray
# vis.color_map('spring') # Options: 'jet' (default), 'hsv', 'hot', 'cool', 'spring', 'summer', 'autumn', 'winter', 'gray'

In [42]:
# # takes a screenshot of current view
vis.capture('screenshot_las.png')

In [21]:
## Load LAZ file
pc_laz = laspy.read('./data/plane.laz')

#pc_las = laspy.convert(pc_laz)

# preparation for visualization
points = np.vstack((pc_laz.x, pc_laz.y, pc_laz.z)).transpose()
print('Shape of Point Cloud:', points.shape)

Shape of Point Cloud: (28185, 3)


In [22]:
vis = pptk.viewer(points) # Start the interactive visualization

### Format: h5 | Package: PPTK

In [23]:
pc_h5 = h5py.File('./data/Scanobjectnn_sample.h5', 'r')
point_clouds = np.array(pc_h5['data'])
labels = np.array(pc_h5['label'])

In [24]:
print('Size of Point Clouds:', point_clouds.shape)
print('labels:', labels)

Size of Point Clouds: (2, 2048, 3)
labels: [ 4 12]


In [25]:
pptk.viewer(point_clouds[0,:,:])

### Format: binary | Package: PPTK

In [26]:
pc_bin = np.fromfile('./data/kitti_007517.bin', '<f4')
pc_bin = np.reshape(pc_bin, (-1, 4))
print(pc_bin.shape)
pptk.viewer(pc_bin[:, :3])

(19669, 4)


### Format: txt | Package: PPTK

In [27]:
pc_txt = np.loadtxt('./data/modelnet40_airplane_0001.txt', delimiter=',')

In [28]:
print('Shape of the point cloud:', pc_txt.shape)
points = pc_txt[:,:3]
colors = pc_txt[:,3:]
vis = pptk.viewer(points, colors) # Start the interactive visualization

Shape of the point cloud: (10000, 6)


### Format: txt | Package: Open3D

In [29]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

In [30]:
# visualization
o3d.visualization.draw_geometries([pcd])